In [2]:
import os


In [3]:
%pwd

'c:\\Users\\Dharshini\\Documents\\kidney-disease\\kidney-disease-prediction\\research'

In [4]:
os.chdir("../")

In [5]:


%pwd

'c:\\Users\\Dharshini\\Documents\\kidney-disease\\kidney-disease-prediction'

In [ ]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path # it'll return the path of this from config.yaml
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


In [7]:


from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf


In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "kidney-ct-scan-image") # inside articraft
        create_directories([
            Path(training.root_dir)
        ])
        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time



In [ ]:
# Import TensorFlow and typing (assumed)
'''ChatGPT said:
Augmentation (in deep learning, especially in image processing) refers to artificially increasing the size and diversity of your training dataset by applying random transformations to the training images. This helps the model generalize better and reduces overfitting.

'''

#Augmentation reduces overfitting by making the model see a more diverse set of training data
import tensorflow as tf
from pathlib import Path
from cnnClassifier.entity.config_entity import TrainingConfig

class Training:
    def __init__(self, config: TrainingConfig):
        """
        Initialize the Training class with a configuration object.
        """
        self.config = config

    
    def get_base_model(self):
        """
        Loads the pre-trained or updated base model from the specified path.
        """
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        """
        Prepares training and validation image data generators.
        Applies augmentation if specified in the config.
        """
        
        # Common preprocessing steps for both train and validation sets
        datagenerator_kwargs = dict(
            rescale=1./255,          # Normalize pixel values
            validation_split=0.20    # Split 20% of data for validation
        )

        # Common parameters for both data generators
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # e.g., [224, 224] if image_size is [224, 224, 3]
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Create a data generator for validation set (no augmentation)
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Load validation images from the directory
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,  # e.g., "artifacts/data_ingestion/kidney-ct-scan-image"
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
       
        # If augmentation is enabled, apply augmentation to training images
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            # If not, use the same preprocessing as validation
            train_datagenerator = valid_datagenerator

        # Load training images from the directory
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the trained model to the specified path.
        """
        model.save(path)

    
    def train(self):
        """
        Trains the model using the training and validation generators.
        """
        # Determine how many steps per epoch based on dataset size
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        # Save the final trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )
#More epochs = more chances to adjust weights and reduce errors.

In [11]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-05-06 11:23:20,489: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-06 11:23:20,496: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-06 11:23:20,498: INFO: common: created directory at: artifacts]
[2025-05-06 11:23:20,501: INFO: common: created directory at: artifacts\training]
Found 93 images belonging to 2 classes.
Found 372 images belonging to 2 classes.
23/23 [==============================] - 307s 13s/step - loss: 9.8171 - accuracy: 0.6039 - val_loss: 32.5475 - val_accuracy: 0.4000
